In [80]:
import os, time
import tensorflow as tf
import imageio
from subpixel import PS
from utils import *

W = 504
H = 378
scale = 3

W1 = 0
W2 = 0
W3 = 0
B1 = 0
B2 = 0
B3 = 0

tf.reset_default_graph()

sess = tf.Session()
with sess.as_default():
    input_LR = tf.placeholder(tf.float32, [None, H, W, 1], name='input')
    feature_tmp = tf.layers.conv2d(input_LR, 64, 5, strides = 1, padding = 'SAME', name = 'CONV_1',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.layers.conv2d(feature_tmp, 32, 3, strides = 1, padding = 'SAME', name = 'CONV_2',
                                kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_out = tf.layers.conv2d(feature_tmp, 1*scale*scale, 3, strides = 1, padding = 'SAME', 
                   name = 'CONV_3', kernel_initializer = tf.contrib.layers.xavier_initializer(), reuse=tf.AUTO_REUSE)
    feature_out = PS(feature_out, 3, color=False)
    output_HR = tf.add(tf.multiply(tf.add(tf.nn.tanh(feature_out),1),(235-16)/2), 16, name = 'output') #Y range is in 16-235
    init_op = tf.global_variables_initializer() 
    
    saver = tf.train.Saver()
    saver.restore(sess, os.path.join("./checkpoint/DIV2K", "ESPCN"))
    #tf.train.write_graph(sess.graph_def, "./checkpoint", 'ESPCN.pb', as_text=False)
    vv = [v for v in tf.global_variables()]
    print("variable list: ", vv)
    W1 = [v for v in tf.global_variables() if v.name == "CONV_1/kernel:0"][0].eval()
    W2 = [v for v in tf.global_variables() if v.name == "CONV_2/kernel:0"][0].eval()
    W3 = [v for v in tf.global_variables() if v.name == "CONV_3/kernel:0"][0].eval()
    B1 = [v for v in tf.global_variables() if v.name == "CONV_1/bias:0"][0].eval()
    B2 = [v for v in tf.global_variables() if v.name == "CONV_2/bias:0"][0].eval()
    B3 = [v for v in tf.global_variables() if v.name == "CONV_3/bias:0"][0].eval()

tf.reset_default_graph()
sess2 = tf.Session()
print("graph reset!")
with sess2.as_default():
    input_LR = tf.placeholder(tf.float32, [None, H, W, 1], name='input')
    feature_tmp = tf.nn.conv2d(input_LR, W1, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B1)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W2, strides=[1, 1, 1, 1], padding='SAME')
    feature_tmp = tf.nn.bias_add(feature_tmp, B2)
    feature_tmp = tf.nn.relu(feature_tmp)
    feature_tmp = tf.nn.conv2d(feature_tmp , W3, strides=[1, 1, 1, 1], padding='SAME')
    feature_out = tf.nn.bias_add(feature_tmp, B3)
    feature_out = PS(feature_out, 3, color=False)
    output_HR = tf.add(tf.multiply(tf.add(tf.nn.tanh(feature_out),1),(235-16)/2), 16, name = 'output') #Y range is in 16-235
    tf.train.write_graph(sess2.graph_def, "./checkpoint", 'ESPCN.pb', as_text=False)
    tf.train.write_graph(sess2.graph_def, "./checkpoint", 'ESPCN.pbtxt', as_text=True)
    LOGDIR='./logs'
    train_writer = tf.summary.FileWriter(LOGDIR)
    train_writer.add_graph(sess2.graph)
    vv = [v for v in tf.global_variables()]
    print("Drew new graph! variable list: ", vv)

INFO:tensorflow:Restoring parameters from ./checkpoint/DIV2K/ESPCN
variable list:  [<tf.Variable 'CONV_1/kernel:0' shape=(5, 5, 1, 64) dtype=float32_ref>, <tf.Variable 'CONV_1/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'CONV_2/kernel:0' shape=(3, 3, 64, 32) dtype=float32_ref>, <tf.Variable 'CONV_2/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'CONV_3/kernel:0' shape=(3, 3, 32, 9) dtype=float32_ref>, <tf.Variable 'CONV_3/bias:0' shape=(9,) dtype=float32_ref>]
graph reset!
Drew new graph! variable list:  []


In [79]:
# Test loading pb file
from tensorflow.core.framework import graph_pb2
tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    #with open('./checkpoint/ESPCN.pb', 'rb') as f:
    #    input_LR = tf.placeholder(tf.float32, [None, H, W, 1], name='input')
    #    sess.graph_def.ParseFromString(f.read())
    #    output = tf.import_graph_def(sess.graph_def, input_map={'input': input_LR}, return_elements=['output'])
    #    output = output[0]
        
    with open('./checkpoint/ESPCN.pb', 'rb') as f:
        input_LR = tf.placeholder(tf.float32, [None, H, W, 1], name='input')
        #graph_def = tf.GraphDef()
        graph_def = graph_pb2.GraphDef()
        graph_def.ParseFromString(f.read())
        output = tf.import_graph_def(graph_def, input_map={'input:0': input_LR}, return_elements=['output:0'])
        output = output[0]
        print(output)

        img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/butterfly.png"), "YCbCr")
        img = doresize(img, [H, W])
        Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
        Y_LR = np.split(img,3, axis=2)[0]
        Cb_LR = np.split(img,3, axis=2)[1]
        Cr_LR = np.split(img,3, axis=2)[2]
        start_time = time.time()
        [out] = sess.run([output], feed_dict={input_LR: [Y_LR]})
        print("runtime: %.6f" %(time.time()-start_time))
        out = np.round(out).astype("uint8")
        print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
        print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
        print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
        save_ycbcr_img(out[0], Cb_LR, Cr_LR, scale, "./test.png")
        save_ycbcr_img(Y_bicubic, Cb_LR, Cr_LR, scale, "./test_bicubic.png")
        #'''

Tensor("import/output:0", shape=(?, 1134, 1512, 1), dtype=float32)
runtime: 0.156057
out size: (1134, 1512, 1) 16 235
out size: (504, 1) 100 123
out size: (504, 1) 134 166


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [14]:
img = load_image(os.path.join("/home/johnyi/deeplearning/research/SISR_Datasets/test/Set5/baby.png"), "YCbCr")
img = doresize(img, [H, W])
Y_bicubic = np.split(imresize(img,[scale*H, scale*W], interp='bicubic'),3,axis=2)[0]
Y_LR = np.split(img,3, axis=2)[0]
Cb_LR = np.split(img,3, axis=2)[1]
Cr_LR = np.split(img,3, axis=2)[2]
start_time = time.time()
[out] = sess.run([output_HR], feed_dict={input_LR: [Y_LR]})
print("runtime: %.6f" %(time.time()-start_time))
out = np.round(out).astype("uint8")
print("out size:",out[0].shape, np.min(out[0]), np.max(out[0]))
print("out size:",Cb_LR[0].shape, np.min(Cb_LR[0]), np.max(Cb_LR[0]))
print("out size:",Cr_LR[0].shape, np.min(Cr_LR[0]), np.max(Cr_LR[0]))
#save_ycbcr_img(out[0], Cb_LR, Cr_LR, scale, "./test.png")
#save_ycbcr_img(Y_bicubic, Cb_LR, Cr_LR, scale, "./test_bicubic.png")

runtime: 0.005027
out size: (1134, 1512, 1) 16 235
out size: (504, 1) 89 129
out size: (504, 1) 125 168


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
